# Co-factor swap

<!-- The balance of co-factors within the cells is important 
In the implementated pathway of progesterone nine NADP and one NAD molecules 
In order for the cell to produce one pregesterone molecule through the implementated pathway nine NADPH molecules and one NAD $^+$ molecules are needed. 
 -->

The balance of co-factors within a cell is important to obtain a high theoretical yield of a given product (https://www.sciencedirect.com/science/article/pii/S1096717614000688?fbclid=IwAR0k-nxrJ6UHrpNyxBcCXlJg58Ioj6N3e5nuFrbhUYAPYHiFNivh4yG3Zds#bib44). Nine NADPH molecules and one NAD $^+$ molecule are needed in order for a cell to produce one progesterone molecule through the implementated pathway. Therefore, by increasing the concentration of avaliable NADPH/NADP $^+$ and NADH/NAD $^+$ co-factors in the the cell, it should be theoretically possible to increase of the yield of progesterone. 
The CofactorSwapOptimization algorithm was used to obtain possible targets

In [1]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [2]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [3]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [5]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [6]:
# Setting the progesterone yield as the formation of progesterone divided by the optake of glucose
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [58]:
# Running the CofactorSwapOptimization Algorithm
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap_1 = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nadp_c","nadph_c"],["nad_c","nadh_c"]),objective_function=py)
print(f'Co-factor swap optimization of the model {model.id} used to increase available {swap_1._metadata["swap_pairs"][0][0]} and {swap_1._metadata["swap_pairs"][0][1]} on the cost of {swap_1._metadata["swap_pairs"][1][0]} and {swap_1._metadata["swap_pairs"][1][1]}')
solution_1 = swap_1.run(max_size = 10)
print("\n")
swap_2 = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nadp_c","nadph_c"],["fad_c","fadh2_m"]),objective_function=py)
print(f'Co-factor swap optimization of the model {model.id} used to increase available {swap_1._metadata["swap_pairs"][0][0]} and {swap_1._metadata["swap_pairs"][0][1]} on the cost of {swap_1._metadata["swap_pairs"][1][0]} and {swap_1._metadata["swap_pairs"][1][1]}')
solution_2 = swap_2.run(max_size = 10)
print("\n")
swap_3 = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["fad_c","fadh2_c"]),objective_function=py)
print(f'Co-factor swap optimization of the model {model.id} used to increase available {swap_1._metadata["swap_pairs"][0][0]} and {swap_1._metadata["swap_pairs"][0][1]} on the cost of {swap_1._metadata["swap_pairs"][1][0]} and {swap_1._metadata["swap_pairs"][1][1]}')
solution_3 = swap_2.run(max_size = 10)

Co-factor swap optimization of the model iMM904_progesterone used to increase available nadp_c and nadph_c on the cost of nad_c and nadh_c
Starting optimization at Mon, 21 Nov 2022 20:13:04


HBox()

Finished after 00:03:44


In [ ]:
# Printing result
print("The following reactions can be used as targets for co-factor swaping:\n")
target_set = set()
for target in solution_1.data_frame.targets:
    for id in target:
        target_set.add(id)
for id in target_set:
    print(model.reactions.get_by_id(id))
print("\n")
print("The following reactions can be used as targets for co-factor swaping:\n")
target_set = set()
for target in solution_2.data_frame.targets:
    for id in target:
        target_set.add(id)
for id in target_set:
    print(model.reactions.get_by_id(id))
print("\n")
print("The following reactions can be used as targets for co-factor swaping:\n")
target_set = set()
for target in solution_3.data_frame.targets:
    for id in target:
        target_set.add(id)
for id in target_set:
    print(model.reactions.get_by_id(id))
print("\n")

The following reaction can be used as targets for co-factor swaping:

R01456: dehydrocholesterol_c + h_c + nadph_c --> cholesterol_c + nadp_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
C4STMO2: 4mzym_c + 3.0 h_c + 3.0 nadph_c + 3.0 o2_c --> 4.0 h2o_c + 3.0 nadp_c + zym_int1_c
ALCD2x_copy2: etoh_c + nad_c <=> acald_c + h_c + nadh_c
GLUDy: glu__L_c + h2o_c + nadp_c <-- akg_c + h_c + nadph_c + nh4_c
C14STR: 44mctr_c + h_c + nadph_c --> 44mzym_c + nadp_c
ALCD2x_copy1: etoh_c + nad_c --> acald_c + h_c + nadh_c
MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
ECYP11A1: cholesterol_c + 6.0 h_c + 6.0 nadph_c + 3.0 o2_c --> 4.0 h2o_c + methylpentanal_c + 6.0 nadp_c + pregnenolone_c
R07215: h_c + lathosterol_c + nadph_c + o2_c --> dehydrocholesterol_c + 2.0 h2o_c + nadp_c
R07498: h_c + nadph_c + zymst_c <=> cholesta8en3betaol_c + nadp_c
C3STKR2: h_c + nadph_c + zym_int2_c --> nadp_c + zymst_c
R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c
C4STMO1: 44mzym_c + 3.0 h_c + 

In [16]:
# Finding the optimal fluxes through each reaction when the objective is biomass production 
Fluxes = model.optimize()
Fluxes